## Load Data

In [ ]:
# path_to_data = '/hpi/fs00/share/MPSS2021BA1/data/'
path_to_data = '../data/arima_results/'

In [ ]:
# 01 - ARIMA - Steady Train Size 12
import pickle
# Heart Rate - First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arima_12_hr_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arima_12_hr_first1000 = pickle.load(input_file)
input_file.close()
# Blood Pressure - First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arima_12_bp_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arima_12_bp_first1000 = pickle.load(input_file)
input_file.close()
# O2 Saturation -  First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arima_12_o2_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arima_12_o2_first1000 = pickle.load(input_file)
input_file.close()

In [ ]:
# 02 - ARIMA - Expanding Train Size 12
import pickle
# Heart Rate - First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arima_expanding_12_hr_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arima_expanding_12_hr_first1000 = pickle.load(input_file)
input_file.close()
# Blood Pressure - First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arima_expanding_12_bp_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arima_expanding_12_bp_first1000 = pickle.load(input_file)
input_file.close()
# O2 Saturation -  First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arima_expanding_12_o2_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arima_expanding_12_o2_first1000 = pickle.load(input_file)
input_file.close()

In [ ]:
# 03 - ARIMAX - Steady Train Size 12
import pickle
# Heart Rate - First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arimax_12_hr_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arimax_12_hr_first1000 = pickle.load(input_file)
input_file.close()
# Blood Pressure - First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arimax_12_bp_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arimax_12_bp_first1000 = pickle.load(input_file)
input_file.close()
# O2 Saturation -  First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arimax_12_o2_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arimax_12_o2_first1000 = pickle.load(input_file)
input_file.close()

In [ ]:
# 04 - ARIMAX - Expanding Train Size 12
import pickle
# Heart Rate - First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arimax_expanding_12_hr_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arimax_expanding_12_hr_first1000 = pickle.load(input_file)
input_file.close()
# Blood Pressure - First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arimax_expanding_12_bp_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arimax_expanding_12_bp_first1000 = pickle.load(input_file)
input_file.close()
# O2 Saturation -  First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arimax_expanding_12_o2_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arimax_expanding_12_o2_first1000 = pickle.load(input_file)
input_file.close()

## Prepare Data

In [ ]:
import pandas as pd
import time

# Transform nested dictionary into data frame using pd.concat()

def dict_to_df(selected_dict):

    starttime = time.time()
    print('START transforming dictionary.')
    chunkno = 0

    selected_dict_as_df = pd.DataFrame({})
    confusion_matrices_per_chunk_in_selected_dict = {}
    for chunkid in selected_dict:
        confusion_matrices_per_chunk_in_selected_dict[chunkid] = pd.concat(selected_dict[chunkid], axis=0).reset_index().rename(columns={'level_0':'ITERATION', 'level_1':'ACCURACY_TYPE'})

        chunkno = chunkno+1
        runningtime = round(((time.time() - starttime) / 60), 5)
        print('Completed chunk '+str(chunkid)+', running time in minutes: '+str(runningtime))

    selected_dict_as_df = pd.concat(confusion_matrices_per_chunk_in_selected_dict, axis=0).reset_index(level=0).rename(columns={'level_0':'CHUNK_ID_FILLED_TH'})

    endtime = round(((time.time() - starttime) / 60), 5)
    print('DONE transforming dictionary.')
    print('Completed '+str(chunkno)+' chunks in '+str(endtime)+' minutes')

    return selected_dict_as_df#.copy()


In [ ]:
# 01 - ARIMA - Steady Train Size 12
# Transform nested dictionary into data frame

confusion_matrices_per_chunk_01_arima_steady_12_hr_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arima_12_hr_first1000)

confusion_matrices_per_chunk_01_arima_steady_12_bp_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arima_12_bp_first1000)

confusion_matrices_per_chunk_01_arima_steady_12_o2_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arima_12_o2_first1000)

In [ ]:
# 02 - ARIMA - Expanding Train Size 12
# Transform nested dictionary into data frame

confusion_matrices_per_chunk_02_arima_expanding_12_hr_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arima_expanding_12_hr_first1000)

confusion_matrices_per_chunk_02_arima_expanding_12_bp_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arima_expanding_12_bp_first1000)

confusion_matrices_per_chunk_02_arima_expanding_12_o2_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arima_expanding_12_o2_first1000)

In [ ]:
# 03 - ARIMAX - Steady Train Size 12
# Transform nested dictionary into data frame

confusion_matrices_per_chunk_03_arimax_steady_12_hr_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arimax_12_hr_first1000)

confusion_matrices_per_chunk_03_arimax_steady_12_bp_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arimax_12_bp_first1000)

confusion_matrices_per_chunk_03_arimax_steady_12_o2_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arimax_12_o2_first1000)

In [ ]:
# 04 - ARIMAX - Expanding Train Size 12
# Transform nested dictionary into data frame

confusion_matrices_per_chunk_04_arimax_expanding_12_hr_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arimax_expanding_12_hr_first1000)

confusion_matrices_per_chunk_04_arimax_expanding_12_bp_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arimax_expanding_12_bp_first1000)

confusion_matrices_per_chunk_04_arimax_expanding_12_o2_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arimax_expanding_12_o2_first1000)

## Analyze Data

This section is not updated yet.

In [ ]:
# 01 - ARIMA - Steady TRAIN Size 12

# Create confusion matrix per chunk by aggregating so that TP, FN, FP, and TN of the several chunk iterations are summed up

# Create confusion matrix by aggregating so that TP, FN, FP, and TN of all chunks are summed up

In [ ]:
# 02 - ARIMA - Expanding TRAIN Size

# Create confusion matrix per chunk by aggregating so that TP, FN, FP, and TN of the several chunk iterations are summed up
confusion_matrix_per_chunk_02_arima_expanding_df = confusion_matrices_per_chunk_02_arima_expanding_df.groupby(['CHUNK_ID_FILLED_TH', 'ACCURACY_TYPE'])[['TP', 'FN', 'FP', 'TN']].agg('sum')
confusion_matrix_per_chunk_02_arima_expanding_df = confusion_matrix_per_chunk_02_arima_expanding_df.reset_index()

# Create confusion matrix by aggregating so that TP, FN, FP, and TN of all chunks are summed up
confusion_matrix_02_arima_expanding_df = confusion_matrix_per_chunk_02_arima_expanding_df.groupby(['ACCURACY_TYPE'])[['TP', 'FN', 'FP', 'TN']].agg('sum')

display(confusion_matrix_02_arima_expanding_df)

In [ ]:
df = confusion_matrix_02_arima_expanding_df
TP_high = df.iloc[0,0]
FN_high = df.iloc[0,1]
FP_high = df.iloc[0,2]
TN_high = df.iloc[0,3]
TP_low = df.iloc[1,0]
FN_low = df.iloc[1,1]
FP_low = df.iloc[1,2]
TN_low = df.iloc[1,3]

fpr_high_02_arima_expanding = round( (FP_high/(FP_high+TN_high)) ,4)
tpr_high_02_arima_expanding = round( (TP_high/(TP_high+FN_high)) ,4) 
fnr_high_02_arima_expanding = round( (FN_high/(TP_high+FN_high)) ,4)
tnr_high_02_arima_expanding = round( (TN_high/(FP_high+TN_high)) ,4)
acc_high_02_arima_expanding = round( ((TP_high+TN_high)/(TP_high+FN_high+FP_high+TN_high)) ,4)
f1s_high_02_arima_expanding = round( (TP_high/(TP_high+0.5*(FP_high+FN_high))) ,4)

fpr_low_02_arima_expanding = round( (FP_low/(FP_low+TN_low)) ,4)
tpr_low_02_arima_expanding = round( (TP_low/(TP_low+FN_low)) ,4) 
fnr_low_02_arima_expanding = round( (FN_low/(TP_low+FN_low)) ,4)
tnr_low_02_arima_expanding = round( (TN_low/(FP_low+TN_low)) ,4)
acc_low_02_arima_expanding = round( ((TP_low+TN_low)/(TP_low+FN_low+FP_low+TN_low)) ,4)
f1s_low_02_arima_expanding = round( (TP_low/(TP_low+0.5*(FP_low+FN_low))) ,4)

print('fpr_high_02_arima_expanding:',fpr_high_02_arima_expanding)
print('tpr_high_02_arima_expanding:',tpr_high_02_arima_expanding)
print('fnr_high_02_arima_expanding:',fnr_high_02_arima_expanding)
print('tnr_high_02_arima_expanding:',tnr_high_02_arima_expanding)
print('acc_high_02_arima_expanding:',acc_high_02_arima_expanding)
print('f1s_high_02_arima_expanding:',f1s_high_02_arima_expanding)
print()
print('fpr_low_02_arima_expanding:',fpr_low_02_arima_expanding)
print('tpr_low_02_arima_expanding:',tpr_low_02_arima_expanding)
print('fnr_low_02_arima_expanding:',fnr_low_02_arima_expanding)
print('tnr_low_02_arima_expanding:',tnr_low_02_arima_expanding)
print('acc_low_02_arima_expanding:',acc_low_02_arima_expanding)
print('f1s_low_02_arima_expanding:',f1s_low_02_arima_expanding)

In [ ]:
# 03 - ARIMAX - Steady TRAIN Size 12

# Create confusion matrix per chunk by aggregating so that TP, FN, FP, and TN of the several chunk iterations are summed up
confusion_matrix_per_chunk_03_arimax_steady_12_df = confusion_matrices_per_chunk_03_arimax_steady_12_df.groupby(['CHUNK_ID_FILLED_TH', 'ACCURACY_TYPE'])[['TP', 'FN', 'FP', 'TN']].agg('sum')
confusion_matrix_per_chunk_03_arimax_steady_12_df = confusion_matrix_per_chunk_03_arimax_steady_12_df.reset_index()

# Create confusion matrix by aggregating so that TP, FN, FP, and TN of all chunks are summed up
confusion_matrix_03_arimax_steady_12_df = confusion_matrix_per_chunk_03_arimax_steady_12_df.groupby(['ACCURACY_TYPE'])[['TP', 'FN', 'FP', 'TN']].agg('sum')

display(confusion_matrix_03_arimax_steady_12_df)

In [ ]:
df = confusion_matrix_03_arimax_steady_12_df
TP_high = df.iloc[0,0]
FN_high = df.iloc[0,1]
FP_high = df.iloc[0,2]
TN_high = df.iloc[0,3]
TP_low = df.iloc[1,0]
FN_low = df.iloc[1,1]
FP_low = df.iloc[1,2]
TN_low = df.iloc[1,3]

fpr_high_03_arimax_steady_12 = round( (FP_high/(FP_high+TN_high)) ,4)
tpr_high_03_arimax_steady_12 = round( (TP_high/(TP_high+FN_high)) ,4) 
fnr_high_03_arimax_steady_12 = round( (FN_high/(TP_high+FN_high)) ,4)
tnr_high_03_arimax_steady_12 = round( (TN_high/(FP_high+TN_high)) ,4)
acc_high_03_arimax_steady_12 = round( ((TP_high+TN_high)/(TP_high+FN_high+FP_high+TN_high)) ,4)
f1s_high_03_arimax_steady_12 = round( (TP_high/(TP_high+0.5*(FP_high+FN_high))) ,4)

fpr_low_03_arimax_steady_12 = round( (FP_low/(FP_low+TN_low)) ,4)
tpr_low_03_arimax_steady_12 = round( (TP_low/(TP_low+FN_low)) ,4) 
fnr_low_03_arimax_steady_12 = round( (FN_low/(TP_low+FN_low)) ,4)
tnr_low_03_arimax_steady_12 = round( (TN_low/(FP_low+TN_low)) ,4)
acc_low_03_arimax_steady_12 = round( ((TP_low+TN_low)/(TP_low+FN_low+FP_low+TN_low)) ,4)
f1s_low_03_arimax_steady_12 = round( (TP_low/(TP_low+0.5*(FP_low+FN_low))) ,4)

print('fpr_high_03_arimax_steady_12:',fpr_high_03_arimax_steady_12)
print('tpr_high_03_arimax_steady_12:',tpr_high_03_arimax_steady_12)
print('fnr_high_03_arimax_steady_12:',fnr_high_03_arimax_steady_12)
print('tnr_high_03_arimax_steady_12:',tnr_high_03_arimax_steady_12)
print('acc_high_03_arimax_steady_12:',acc_high_03_arimax_steady_12)
print('f1s_high_03_arimax_steady_12:',f1s_high_03_arimax_steady_12)
print()
print('fpr_low_03_arimax_steady_12:',fpr_low_03_arimax_steady_12)
print('tpr_low_03_arimax_steady_12:',tpr_low_03_arimax_steady_12)
print('fnr_low_03_arimax_steady_12:',fnr_low_03_arimax_steady_12)
print('tnr_low_03_arimax_steady_12:',tnr_low_03_arimax_steady_12)
print('acc_low_03_arimax_steady_12:',acc_low_03_arimax_steady_12)
print('f1s_low_03_arimax_steady_12:',f1s_low_03_arimax_steady_12)

In [ ]:
# 04 - ARIMAX - Expanding TRAIN Size

# Create confusion matrix per chunk by aggregating so that TP, FN, FP, and TN of the several chunk iterations are summed up

# Create confusion matrix by aggregating so that TP, FN, FP, and TN of all chunks are summed up

In [ ]:
# Comparison
comparison_df = pd.DataFrame({
    'ID': [
        '02', '02',
        '03', '03'
        ],
    'PARAMETER': [
        'HR', 'HR',
        'HR', 'HR'
        ],
    'MODEL': [
        'ARIMA', 'ARIMA',
        'ARIMAX', 'ARIMAX'
        ],
    'RESAMPLING': [
        'Median', 'Median',
        'Max', 'Min'
        ],
    'EXOGENOUS': [
        '', '',
        'Median', 'Median'
        ],
    'TRAINSIZE': [
        'Expanding', 'Expanding',
        'Steady 12h', 'Steady 12h'
        ],
    'ALARMTYPE': [
        'High', 'Low',
        'High', 'Low'
        ],
    'FPR': [
        fpr_high_02_arima_expanding, fpr_low_02_arima_expanding,
        fpr_high_03_arimax_steady_12, fpr_low_03_arimax_steady_12
        ],
    'TPR': [
        tpr_high_02_arima_expanding, tpr_low_02_arima_expanding,
        tpr_high_03_arimax_steady_12, tpr_low_03_arimax_steady_12
        ],
    'FNR': [
        fnr_high_02_arima_expanding, fnr_low_02_arima_expanding,
        fnr_high_03_arimax_steady_12, fnr_low_03_arimax_steady_12
        ],
    'TNR': [
        tnr_high_02_arima_expanding, tnr_low_02_arima_expanding,
        tnr_high_03_arimax_steady_12, tnr_low_03_arimax_steady_12
        ],
    'ACC': [
        acc_high_02_arima_expanding, acc_low_02_arima_expanding,
        acc_high_03_arimax_steady_12, acc_low_03_arimax_steady_12
        ],
    'F1S': [
        f1s_high_02_arima_expanding, f1s_low_02_arima_expanding,
        f1s_high_03_arimax_steady_12, f1s_low_03_arimax_steady_12
        ]
    })

In [ ]:
display(comparison_df)

## Comparison for selected Chunk

In [ ]:
selected_chunkid = '200347.0_220045.0_2116-06-05 15:02:00'
selected_iteration = 11

In [ ]:
# 02 - ARIMA - Expanding TRAIN Size
# Confusion matrix for selected chunk
confusion_matrix_per_chunk_02_arima_expanding_df[
    confusion_matrix_per_chunk_02_arima_expanding_df.CHUNK_ID_FILLED_TH == selected_chunkid]

In [ ]:
# 02 - ARIMA - Expanding TRAIN Size
# Confusion matrix for selected iteration for selected chunk
confusion_matrices_per_chunk_02_arima_expanding_df[
    (confusion_matrices_per_chunk_02_arima_expanding_df.CHUNK_ID_FILLED_TH == selected_chunkid) &
    (confusion_matrices_per_chunk_02_arima_expanding_df.ITERATION == selected_iteration)
    ]

In [ ]:
# 03 - ARIMAX - Steady TRAIN Size 12
# Confusion matrix for selected chunk
confusion_matrix_per_chunk_03_arimax_steady_12_df[
    confusion_matrix_per_chunk_03_arimax_steady_12_df.CHUNK_ID_FILLED_TH == selected_chunkid]

In [ ]:
# 03 - ARIMAX - Steady TRAIN Size 12
# Confusion matrix for selected iteration for selected chunk
confusion_matrices_per_chunk_03_arimax_steady_12_df[
    (confusion_matrices_per_chunk_03_arimax_steady_12_df.CHUNK_ID_FILLED_TH == selected_chunkid) &
    (confusion_matrices_per_chunk_03_arimax_steady_12_df.ITERATION == selected_iteration)
    ]